<h4>Librerías</h4>

In [22]:
import os
import arcpy
import pathlib
import geopandas as gpd
import fiona
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

<h4>Creación de base de datos y parametrización de rutas</h4>

In [23]:
# Ruta del directorio donde se localizan los archivos geojson.
ruta_geojson = r'D:\PUBLIC\3_Camara_Medellin\3_1_Insumos\3_1_1_GeoMedellin\zGeoJSON'
# Espacio de trabajo, asociada a la base de datos donde se almanacenarán los geojson convertidos a feature class
arcpy.env.workspace = r"D:\PUBLIC\3_Camara_Medellin\3_2_Resultados\3_2_2_BD\Insumos_GeoMedellin.gdb"

# Borrado de capas dentro de la base de datos existente
if arcpy.Exists(arcpy.env.workspace):
    for capas in arcpy.ListFeatureClasses():
        arcpy.management.Delete(capas)
        print("Se borra la información registrada en la capa: {0}".format(capas))
else:
    print("No hay capas alojadas en la base de datos")

Se borra la información registrada en la capa: establecimientos_industria_comercio
Se borra la información registrada en la capa: estaciones_transporte_masivo
Se borra la información registrada en la capa: limite_comuna_corregimiento
Se borra la información registrada en la capa: lineas_sistema_transporte
Se borra la información registrada en la capa: observatorio_inmobiliario
Se borra la información registrada en la capa: paradas_transporte_publico
Se borra la información registrada en la capa: rutas_unificadas_transporte_publico


<h4>Parametrización de las rutas donde se localizan los .geojson</h4>

In [24]:
nombre_geojson = os.listdir(ruta_geojson)
lista_ruta_archivo = []
for rutas in nombre_geojson:
    lista_ruta_archivo.append(os.path.join(ruta_geojson, rutas))   

<h4>Conversión de .geojson a Feature Class</h4>

In [25]:
# ? Diccionario que se usará en la transformación del geoJSON a Feature Class
geometrias = {'Polygon':'POLYGON',
              'MultiPolygon':'POLYGON',
              'LineString':'POLYLINE',
              'MultiLineString':'POLYLINE',
              'Point':'POINT'}

for generacion_feature in lista_ruta_archivo:
    if pathlib.Path(generacion_feature).suffix == '.geojson':
        nombre_geojson = os.path.split(generacion_feature)
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace(" ","_")
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace("-","_")
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace("%","_")
        if nombre_geojson_parametrizado != os.path.basename(ruta_geojson):
            salida_feature = os.path.join(arcpy.env.workspace, nombre_geojson_parametrizado)
            # ? Se exporta el geoJSON a GeoDataFrame
            df = gpd.read_file(generacion_feature)
            # ? Por medio del atributo .geom_type se extrae el tipo de geometría para cada registro
            tipo_geometria = df.geom_type
            # ? Como las geometrias varías si son simples o multiples, se seleccina la que se almacena en la primera fila
            primer_registro_tipo_geometria = tipo_geometria.loc[0]
            # ? Se recorre el diccionario
            for valor, llave in geometrias.items():
                # ? Se compara la geometría seleccionada con el valor almacenado en el diccionario
                if primer_registro_tipo_geometria == valor:
                    # ? Se genera el feature class, tomando como variable el valor de la geometría
                    arcpy.conversion.JSONToFeatures(generacion_feature, salida_feature, llave)
                    print("geojson {0} convertido al feature {1}".format(nombre_geojson, nombre_geojson_parametrizado))
    elif pathlib.Path(generacion_feature).suffix == '.json':
        nombre_geojson = os.path.split(generacion_feature)
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace(" ","_")
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace("-","_")
        nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace("%","_")
        if nombre_geojson_parametrizado != os.path.basename(ruta_geojson):
            salida_feature = os.path.join(arcpy.env.workspace, nombre_geojson_parametrizado)
            # ? Se exporta el geoJSON a GeoDataFrame
            df = gpd.read_file(generacion_feature)
            # ? Por medio del atributo .geom_type se extrae el tipo de geometría para cada registro
            tipo_geometria = df.geom_type
            # ? Como las geometrias varías si son simples o multiples, se seleccina la que se almacena en la primera fila
            primer_registro_tipo_geometria = tipo_geometria.loc[0]
            # ? Se recorre el diccionario
            for valor, llave in geometrias.items():
                # ? Se compara la geometría seleccionada con el valor almacenado en el diccionario
                if primer_registro_tipo_geometria == valor:
                    # ? Se genera el feature class, tomando como variable el valor de la geometría
                    arcpy.conversion.JSONToFeatures(generacion_feature, salida_feature, llave)
                    print("json {0} convertido al feature {1}".format(nombre_geojson, nombre_geojson_parametrizado))
    else:
        print("Es la base de datos")  

geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'areas_declaratoria_proteccion.geojson') convertido al feature areas_declaratoria_proteccion
geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'ciclorutas.geojson') convertido al feature ciclorutas
geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'clasificacion_suelo.geojson') convertido al feature clasificacion_suelo
geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'corredores_transporte_pasajeros.geojson') convertido al feature corredores_transporte_pasajeros
geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'establecimientos_industria_comercio.geojson') convertido al feature establecimientos_industria_comercio
geojson ('D:\\PUBLIC\\3_Camara_Medellin\\3_1_Insumos\\3_1_1_GeoMedellin\\zGeoJSON', 'estaciones_transporte_masivo.geojson') convertido al feature estaciones_

<h4>Creación y registro de nombre del feature class en cada una de las capas convertidas</h4>

In [26]:
lista_capas_convertidas = arcpy.ListFeatureClasses()
nombre_campo = 'nombre_capa'

for features in lista_capas_convertidas:
    descripcion = arcpy.Describe(features)
    expresion = "'" + descripcion.name +"'"
    arcpy.management.AddField(features, nombre_campo, 'TEXT', field_length = 100, field_alias = 'Nombre Capa Origen')
    arcpy.management.CalculateField(features, nombre_campo, expresion, expression_type = 'PYTHON3')
    print("Se actualiza la capa {0}".format(features))

Se actualiza la capa areas_declaratoria_proteccion
Se actualiza la capa ciclorutas
Se actualiza la capa clasificacion_suelo
Se actualiza la capa corredores_transporte_pasajeros
Se actualiza la capa establecimientos_industria_comercio
Se actualiza la capa estaciones_transporte_masivo
Se actualiza la capa estrato_socioeconomico
Se actualiza la capa limite_comuna_corregimiento
Se actualiza la capa lineas_sistema_transporte
Se actualiza la capa lotes
Se actualiza la capa manzana_catastral
Se actualiza la capa observatorio_inmobiliario
Se actualiza la capa paradas_transporte_publico
Se actualiza la capa proyeccion_hogares_2018_2030
Se actualiza la capa proyeccion_viviendas_totales_2018_2030
Se actualiza la capa red_peatonal
Se actualiza la capa rutas_unificadas_transporte_publico
Se actualiza la capa suelo_proteccion_para_la_produccion
Se actualiza la capa usos_generales_suelo_urbano
Se actualiza la capa usos_predio
Se actualiza la capa vias_urbano_rural
Se actualiza la capa zonas_homogenea